# Ag-Analytics® - POLARIS Soils API 

Polaris is a recently developed soil data set that provides additional soil attributes. The POLARIS Soils Data Layers API provides a spatially continuous, internally consistent, quantitative prediction of soil series at a 30 m spatial resolution for the contiguous United States.

In [6]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


Request URL:  https://ag-analytics.azure-api.net/polaris-new/

1). __aoi__ (.tif, GeoJSON String): Area of interest as either TIFF file or Geojson string containing boundary coordinates.
       
2). __Soil_Parameter__ (string): Soil parameter to return.
    
    silt - silt percentage, %
    sand - sand percentage, %
    clay - clay percentage, %
    bd - bulk density, g/cm3
    awc - available water content, m3/m3
    theta_s - saturated soil water content, m3/m3
    theta_r - residual soil water content, m3/m3
    theta_33 - soil water content at field capacity, m3/m3
    theta_1500 - soil water content at the wilting point, m3/m3
    ksat - saturated hydraulic conductivity, cm/hr
    resdt - depth to restriction layer, cm
    ph - soil pH in H2O, N/A
    om - organic matter, %
    caco3 - calcium carbonate in soil, %
    cec - cation exchange capacity of soil, meq/100g
    lambda - pore size distribution index (brooks-corey), N/A
    hb - bubbling pressure (brooks-corey), cm
    n - measure of the pore size distribution (van genuchten), N/A
    alpha - scale parameter inversely proportional to mean pore diameter (van genuchten), cm-1

3). __Depth_Range__ (string): Depth in soil column.

    0-5 cm
    5-15 cm
    15-30 cm
    30-60 cm
    60-100 cm
    100-200 cm

4). __Statistic__ (string): Soil parameter statistic to return.

    mean - Arithmetic mean
    min - Minimum
    max - Maximum
    var - Variance

---
## POST Request API Fuction

In [18]:
def polaris_service(values, files, headers):
    try:
        url = 'https://ag-analytics.azure-api.net/polaris-new'
        
        if files == None:
            response = requests.post(url, data=values).json()
        else:
            response = requests.post(url, files = files, data = values).json()
            
        print(response)
        
        return response
    
    except Exception as e:
        print(e)
        raise e

## Raster (.tif) as AOI

In [34]:
#Path to raster in your local machine
rasterpath = r"C:\Users\User\Documents\Blobs\Harvest_Rasters\raster_VRYIELDVOL_0-0001.tif"
gtiffs_files = [('Raster_File',(open(rasterpath, 'rb')))]
print(gtiffs_files)

#Parameters to create index
gtiffs_values = {"Soil_Parameter": "clay",
            "Depth_Range": "15-30",
            "Statistic": "mean"}

# Header for using a subscription key.
gtiffs_headers=None
# headers={'content-type': "application/json",'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

[('Raster_File', <_io.BufferedReader name='C:\\Users\\Alex\\Documents\\Blobs\\Harvest_Rasters\\e036f472-7a0b-4d77-9ed7-5e3e91cc150f\\raster_VRYIELDVOL_0-0001.tif'>)]


In [35]:
IndexResponse = polaris_service(gtiffs_values, gtiffs_files, gtiffs_headers)

{'Features': [{'attributes': {'CellSize': [4.776798576804391e-05, -4.776798576804391e-05], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-89.7589908770968, 40.45595637048064, -89.75039263965856, 40.46455460791889', 'Legend': [{'Area': '6.72 %', 'Count': 833, 'CountAllPixels': 12400, 'Max': 14.337621688842773, 'Mean': 13.170328617095947, 'Min': 12.003035545349121, 'color': '#d305fc'}, {'Area': '17.46 %', 'Count': 2165, 'CountAllPixels': 12400, 'Max': 16.672207832336426, 'Mean': 15.5049147605896, 'Min': 14.337621688842773, 'color': '#e570ff'}, {'Area': '27.23 %', 'Count': 3377, 'CountAllPixels': 12400, 'Max': 19.006793975830078, 'Mean': 17.839500904083252, 'Min': 16.672207832336426, 'color': '#eb94ff'}, {'Area': '20.99 %', 'Count': 2603, 'CountAllPixels': 12400, 'Max': 21.34138011932373, 'Mean': 

---
## GeoJSON as AOI

In [28]:
#Parameters to create index
gjson_values = {"aoi": "{\"type\":\"Feature\",\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-121.2475204, 45.4668127],[-121.2484646, 45.4418262],[-121.2119007, 45.4417660],[-121.2115574, 45.4665117],[-121.2475204, 45.4668127]]]}}",
            "Soil_Parameter": "ph",
            "Depth_Range": "15-30",
            "Statistic": "mean",
            "Legend_Ranges": "10"}

gjson_files = None

# Header for using a subscription key.
gjson_headers = {'content-type': "application/json"}
# headers={'content-type': "application/json",'Ocp-Apim-Subscription-Key': "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"}

In [12]:
IndexResponse = polaris_service(gjson_values, gjson_files, gjson_headers)

{'Features': [{'attributes': {'CellSize': [5.639292608437955e-05, -5.639292608437955e-05], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'Extent': '-121.24928150978623, 45.440926248038664, -121.21093432004885, 45.46686699403748', 'Legend': [{'Area': '0.3 %', 'Count': 922, 'CountAllPixels': 307200, 'Max': 6.144077634811401, 'Mean': 6.092171359062195, 'Min': 6.040265083312988, 'color': '#d305fc'}, {'Area': '1.21 %', 'Count': 3718, 'CountAllPixels': 307200, 'Max': 6.247890186309815, 'Mean': 6.195983910560608, 'Min': 6.144077634811401, 'color': '#d71ffc'}, {'Area': '8.26 %', 'Count': 25379, 'CountAllPixels': 307200, 'Max': 6.351702737808227, 'Mean': 6.299796462059021, 'Min': 6.247890186309815, 'color': '#dc3afd'}, {'Area': '26.74 %', 'Count': 82157, 'CountAllPixels': 307200, 'Max': 6.455515289306641, 'Mean': 

## Visualize the response png

In [36]:
df=defaultdict(list)
for value in IndexResponse['Features']:
    attributes = value['attributes']
    df['Index'].append(attributes['Product'])
    df['Max'].append(float(attributes['Max']))
    df['Mean'].append(float(attributes['Mean']))
    df['Min'].append(float(attributes['Min']))
    df['pngb64'].append(attributes['pngb64'])

indexdf=pd.DataFrame.from_dict(df)
indexdf

,Index,Max,Mean,Min,pngb64
0,clay,23.675966,18.872132,12.003036,"data:image/png;base64, iVBORw0KGgoAAAANSUhEUgA..."


In [37]:
#Image Legend
df=defaultdict(list)
for product in IndexResponse['Features']:
    Legend = product['attributes']['Legend']
    
for val in Legend:
    df['Max'].append(float(val['Max']))
    df['Mean'].append(float(val['Mean']))
    df['Min'].append(float(val['Min']))
    df['Color'].append(val['color'])
#     df['Range'] = ['Low','Medium','High']
    
legenddf = pd.DataFrame.from_dict(df)    
legenddf.style.applymap(lambda x:"background-color: %s"%x, subset=['Color'])

,Max,Mean,Min,Color
0,14.3376,13.1703,12.003,#d305fc
1,16.6722,15.5049,14.3376,#e570ff
2,19.0068,17.8395,16.6722,#eb94ff
3,21.3414,20.1741,19.0068,#f2b9ff
4,23.676,22.5087,21.3414,#f9deff


In [38]:
# IndexImage = indexdf.loc[indexdf['Index'] == 'POLARIS_Soils','pngb64'].iloc[0]
IndexImage = IndexResponse["Features"][0]['attributes']["pngb64"]
Image(url = IndexImage, width = 500, height = 500)